# BIC

In [1]:
from pathlib import Path
import os

os.chdir(Path(os.getcwd()).parent)

In [2]:
from AnomalyCableDetection.load import Loader, Preprocessor
from AnomalyCableDetection.stl import CableSTL, CrossCorrelation, AdjacencyType
from AnomalyCableDetection.gmm import *
from AnomalyCableDetection.plot import *

In [3]:
from sklearn import mixture
from os.path import join
from pathlib import Path

import matplotlib.dates as mdates
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np
import glob
import os
import re

In [4]:
hour = 10

In [5]:
gmm = CableGMM(hourly=True, hour=hour)
cable_dict = gmm.cable_dict
date_list = gmm.date_list
scatter_dict = gmm.scatter_dict
cable_list = get_dup_cable_numbers(cable_dict[date_list[0]])

```python
bic_path = join(Path(os.getcwd()), 'results', 'gmm', 'bic_hourly')

best_cluster_list = []
for cable_number in cable_list:
    bic_df, best_cluster = gmm.get_cable_bic_set(cable_number)
    bic_df.to_csv(join(bic_path, f'SJ{str(cable_number).zfill(2)}_{hour}.csv'))
    best_cluster_list.append(best_cluster)
```

In [6]:
bic_path = join(Path(os.getcwd()), 'results', 'gmm', 'bic_hourly')
path_list = glob.glob(join(bic_path, f'*_{hour}.csv'))
cable_regex = r'[A-X]{2}\d{2}'

cable_names = []
bic_dict = dict()
for path in path_list:
    print(path)
    m = re.search(cable_regex, path)
    cable_name = m.group()
    cable_names.append(cable_name)
    bic_df = pd.read_csv(path, index_col=0)
    bic_dict.update({cable_name: bic_df})

/home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/SJ09_10.csv
/home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/SJ10_10.csv
/home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/SJ11_10.csv
/home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/SJ12_10.csv
/home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/SJ14_10.csv


In [7]:
tmp_df = bic_dict[cable_names[0]]
columns = tmp_df.columns.tolist()

mean_df = pd.DataFrame(index=cable_names, columns=columns)

for cable_name in bic_dict:
    bic_df = bic_dict[cable_name]
    mean = bic_df.loc['mean',:].to_list()
    mean_df.loc[cable_name, :] = mean
    
mean_df

,1,2,3,4,5,6,7,8,9
SJ09,101.047312,97.442782,107.462648,116.727166,126.658076,137.818926,147.405554,157.471373,167.497456
SJ10,90.56338,87.576672,97.156126,109.335348,120.099361,129.421487,140.122169,150.090554,160.649159
SJ11,79.390347,75.247794,88.008405,99.312809,108.742882,119.956171,129.828465,140.130158,150.556695
SJ12,74.82438,67.813485,74.806951,86.280701,97.587505,107.186279,117.65655,129.303911,140.897012
SJ14,55.771414,48.765423,59.531989,71.703425,83.209271,94.028536,102.217071,113.825666,123.898189


In [8]:
mean_path = join(bic_path, f'mean_{hour}.csv')
mean_df.to_csv(mean_path)
print(f'saving mean_df to {mean_path}')

saving mean_df to /home/vislab/Github/AnomalyCableDetection/results/gmm/bic_hourly/mean_10.csv


In [9]:
for index, row in mean_df.iterrows():
    row = row.to_list()
    best_cluster = row.index(min(row)) + 1
    print(index, best_cluster)

SJ09 2
SJ10 2
SJ11 2
SJ12 2
SJ14 2
